In [14]:
"""
Frank Chien
BMI 500 Week 6, NLP
Vectorizing and part of speech tagging
"""

import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#1a: opening, reading, and lower casing the two provided files
abstract1 = open('./text1').read().lower()
abstract2 = open('./text2').read().lower()
corpus = [abstract1, abstract2]
print(corpus)

['in a prospective, double-blind, randomized trial the efficacy of a heparinoid in ointment form was assessed in treating superficial thrombophlebitis developing after continuous intravenous infusion. one hundred surgical patients were studied, and clinical examination and the iodine-125-labelled fibrinogen test used to assess the results. the mean time required for the relief of local symptoms and signs and the rate of local decline in radioactivity differed significantly between patients receiving the heparinoid cream and those recieving the placebo.', 'in a randomized, double-blind five-year trial, we tested the efficacy of simultaneously elevating serum levels of high-density lipoprotein (hdl) cholesterol and lowering levels of non-hdl cholesterol with gemfibrozil in reducing the risk of coronary heart disease in 4081 asymptomatic middle-aged men (40 to 55 years of age) with primary dyslipidemia (non-hdl cholesterol greater than or equal to 200 mg per deciliter [5.2 mmol per liter]

In [7]:
#1b
vectorizer = CountVectorizer(ngram_range=(1,3))
vectorized_corpus = vectorizer.fit_transform(corpus)
print(vectorized_corpus)


  (0, 285)	4
  (0, 511)	1
  (0, 196)	1
  (0, 108)	1
  (0, 517)	1
  (0, 698)	1
  (0, 619)	8
  (0, 203)	1
  (0, 430)	3
  (0, 274)	2
  (0, 456)	1
  (0, 233)	1
  (0, 722)	1
  (0, 91)	1
  (0, 692)	1
  (0, 591)	1
  (0, 669)	1
  (0, 175)	1
  (0, 48)	1
  (0, 146)	1
  (0, 339)	1
  (0, 333)	1
  (0, 459)	1
  (0, 282)	1
  (0, 594)	1
  :	:
  (1, 562)	1
  (1, 87)	1
  (1, 289)	1
  (1, 47)	1
  (1, 747)	1
  (1, 716)	1
  (1, 507)	1
  (1, 705)	1
  (1, 739)	1
  (1, 189)	1
  (1, 492)	1
  (1, 59)	1
  (1, 68)	1
  (1, 329)	1
  (1, 618)	1
  (1, 420)	1
  (1, 446)	1
  (1, 372)	1
  (1, 362)	1
  (1, 743)	1
  (1, 245)	1
  (1, 544)	1
  (1, 195)	1
  (1, 295)	1
  (1, 403)	1


In [8]:

#1c Computer cosign similarity
co_sim = cosine_similarity(vectorized_corpus)
print(co_sim)


[[1.        0.3974542]
 [0.3974542 1.       ]]


In [9]:
#1d
# to do jaccard distance, first the corpus needs to be tokenized
tokenized_abs1 = set(abstract1.split())
tokenized_abs2 = set(abstract2.split())
print("jaccard distance is", nltk.jaccard_distance(tokenized_abs1, tokenized_abs2))


jaccard distance is 0.9251336898395722


In [123]:
#2A
#Importing Brown Corpus
#to access brown corups, the following was entered into terminal:
#python -m nltk.downloader all
#corpus was saved onto /Users/frankchien/nltk_data...
#some coding examples obtained from https://www.nltk.org/book/ch05.html

from nltk.corpus import brown
from pickle import dump
brown_tagged_sents = brown.tagged_sents(categories='fiction')

freq_dist = nltk.FreqDist(brown.words(categories='fiction'))
cond_freq_dist = nltk.ConditionalFreqDist(brown.tagged_words(categories='fiction'))

#separating the training set and the testing set
train_test_cutoff = int(len(brown_tagged_sents)*0.9) #sets cutoff 90% through dataset
train_set = brown_tagged_sents[:train_test_cutoff] #gets first 90% of dataset
test_set = brown_tagged_sents[train_test_cutoff:] #gets last 10% of dataset

def dumb_tagger(): #everything is a awesome, and a noun
    default_tagger = nltk.DefaultTagger('NN')
    print("Accuracy of default tagger: %4.3f %%" %(100*default_tagger.evaluate(brown_tagged_sents)))
    
def unigram_tagger(n):
    """unigram tagging, ie look-up tagger. This is not context specific, so would give the same answer to 'plant' regardless
    of whether its used in 'to plant' or 'a plant'
    """
    print("Using n= %d unigram tagger" %n)
    most_common_tokens = freq_dist.most_common(n)
    likely_tags = dict((word, cond_freq_dist[word].max()) for (word, _) in most_common_tokens)
    baseline_tagger = nltk.UnigramTagger(model=likely_tags)
    baseline_backoff_tagger = nltk.UnigramTagger(model=likely_tags,backoff=nltk.DefaultTagger('NN'))
    baseline_backoff_tagger = nltk.UnigramTagger(model=likely_tags,backoff=nltk.DefaultTagger('NN'))
    """this code will allow you to see the tagging of each individual word. For example purposes, the 3rd sentences is used
    sents = brown.sents(categories='fiction')[3]
    print(baseline_tagger.tag(sents)) #allows you to visualize the tags. Those that were not seen in first n words gets "None"
    """

    print("Accuracy of unigram tagger of %d most freq words: %4.3f %%" %(n, 100*baseline_tagger.evaluate(brown_tagged_sents))) 
    print("Accuracy of unigram tagger of %d most freq words and defaulting to nouns: %4.3f %%" %(n, 100*baseline_backoff_tagger.evaluate(brown_tagged_sents)))

def bigram_tagger(training_set,test_set):
    print ("Using bigrams")
    bigram_tagger = nltk.BigramTagger(training_set)
    print("Performance of bigram tagger: %4.3f %%" %(100*bigram_tagger.evaluate(test_set)))

def ultimate_tagger(training_set, test_set, n):
    #this function will also pickle the combination tagger
    print ("Using combination tagging strategy using bigram tagger, then backoff to unigram, then backoff to noun default")
    most_common_tokens = freq_dist.most_common(n)
    likely_tags = dict((word, cond_freq_dist[word].max()) for (word, _) in most_common_tokens)
    baseline_tagger = nltk.UnigramTagger(model=likely_tags)
    
    unigram_backoff_tagger = nltk.UnigramTagger(model=likely_tags,backoff=nltk.DefaultTagger('NN'))
    ultimate_tagger = nltk.BigramTagger(training_set, backoff=unigram_backoff_tagger)
    print("Performance of combination tagger: %4.3f %% " %(100*ultimate_tagger.evaluate(test_set)))
    
    #pickling ultimate_tagger
    output = open('ultimate_tagger.pkl', 'wb')
    dump(ultimate_tagger, output, -1)
    output.close()


print("===================================")
dumb_tagger()
print("===================================")
unigram_tagger(50)
unigram_tagger(200)
unigram_tagger(1000)
print("===================================")
bigram_tagger(train_set,test_set)
print("===================================")
ultimate_tagger(train_set,test_set,1000)


Accuracy of default tagger: 11.411 %
Using n= 50 unigram tagger
Accuracy of unigram tagger of 50 most freq words: 45.588 %
Accuracy of unigram tagger of 50 most freq words and defaulting to nouns: 56.998 %
Using n= 200 unigram tagger
Accuracy of unigram tagger of 200 most freq words: 59.152 %
Accuracy of unigram tagger of 200 most freq words and defaulting to nouns: 69.195 %
Using n= 1000 unigram tagger
Accuracy of unigram tagger of 1000 most freq words: 73.828 %
Accuracy of unigram tagger of 1000 most freq words and defaulting to nouns: 79.702 %
Using bigrams
Performance of bigram tagger: 16.743 %
Using combination tagging strategy using bigram tagger, then backoff to unigram, then backoff to noun default
Performance of combination tagger: 83.799 % 


In [104]:
"""
using NLTK tagger on text1 from homework assignment
"""
from pickle import load

input = open('ultimate_tagger.pkl', 'rb')
ultimate_tagger = load(input)
input.close()

abstract1 = open('./text1').read().split()
ultimate_tagger.tag(abstract1)


[('In', 'IN'),
 ('a', 'AT'),
 ('prospective,', 'NN'),
 ('double-blind,', 'NN'),
 ('randomized', 'NN'),
 ('trial', 'NN'),
 ('the', 'AT'),
 ('efficacy', 'NN'),
 ('of', 'IN'),
 ('a', 'AT'),
 ('heparinoid', 'NN'),
 ('in', 'IN'),
 ('ointment', 'NN'),
 ('form', 'NN'),
 ('was', 'BEDZ'),
 ('assessed', 'NN'),
 ('in', 'IN'),
 ('treating', 'NN'),
 ('superficial', 'NN'),
 ('thrombophlebitis', 'NN'),
 ('developing', 'NN'),
 ('after', 'IN'),
 ('continuous', 'NN'),
 ('intravenous', 'NN'),
 ('infusion.', 'NN'),
 ('One', 'CD'),
 ('hundred', 'CD'),
 ('surgical', 'NN'),
 ('patients', 'NN'),
 ('were', 'BED'),
 ('studied,', 'NN'),
 ('and', 'CC'),
 ('clinical', 'NN'),
 ('examination', 'NN'),
 ('and', 'CC'),
 ('the', 'AT'),
 ('iodine-125-labelled', 'NN'),
 ('fibrinogen', 'NN'),
 ('test', 'NN'),
 ('used', 'VBN'),
 ('to', 'TO'),
 ('assess', 'NN'),
 ('the', 'AT'),
 ('results.', 'NN'),
 ('The', 'AT'),
 ('mean', 'JJ'),
 ('time', 'NN'),
 ('required', 'NN'),
 ('for', 'IN'),
 ('the', 'AT'),
 ('relief', 'NN'),
 ('of'

In [ ]:
"""
can lowercasing affect the performance?

Yes, because in natural language, casing can give information with regards to parts of speech. This happens with proper nouns.
Examples include: Seattle Seahawks, Philadelphia Eagles, or Venmo. 
In cases such as Venmo or Google, capitalization can change the part of speech. 
Consider "may I venmo you the money?". Here venmo is used as a verb
Alternatively, consider "I've read that Venmo is owned by PayPal". Here, Venmo refers to the app and is a noun.

Thus, lowercasing all tokens in the a text can effect POS tagger performance because you may lose information on whether or 
not a token was used referring to the action it represents or as a proper noun.
"""

In [ ]:
"""
For question 3 - see the attached write up pdf
"""